# 12조 중간보고서

**주제**: 공공자전거 데이터 분석을 통한 자전거 재배치 최소화

**조원**: 201904008 곽재원 202104216 백종민

## 개요

저희 조는 날씨와 대여 이력 간의 관계, 대여소의 위치와 대여 이력 간의 관계를 통해 "최적의 공공자전거 재배치 및 동선 설계"을 목표로 두고 있습니다.

중간 보고서에서 데이터를 불러온 후 그 데이터들의 구조와 형태를 알아보고 불필요한 데이터 제거 및 통합을 토대로 가용적인 정보로 만드는 것을 진행하고자 하며,

최종 보고서에서 데이터를 활용하여 최적의 공공자전거 재배치 동선을 설계하고자 합니다.


In [1]:
# 중간보고서 전반에 걸쳐 공통적으로 사용되는 라이브러리들을 로드한다.
import pandas as pd
import numpy as np
import os

# 경로를 설정하고 해당 작업환경으로 이동한다.
work_dir = '/Users/jaewone/Downloads/공공자전거'
data_path = os.path.join(work_dir, 'data')
location_info_path = os.path.join(data_path, 'locationInfo')
preprocessing_path = os.path.join(data_path, 'preprocessing')
os.chdir(work_dir)

# 폴더가 없을 경우 생성한다.
if not os.path.exists(location_info_path):
    os.mkdir(location_info_path)
if not os.path.exists(preprocessing_path):
    os.mkdir(preprocessing_path)

## 데이터 세트 가져오기

날씨와 대여 이력 간의 관계, 대여소의 위치와 대여 이력 간의 관계를 분석하기 위해 아래 3개의 데이터 세트를 사용할 것입니다.

- **서울시 공공자전거 대여 이력**

- **서울시 공공자전거 대여소**

- **종관기상관측(ASOS) 자료**


### "서울시 공공자전거 대여 이력" 정보 수집

본 데이터 세트의 https://data.seoul.go.kr/dataList/OA-15182/F/1/datasetView.do 홈페이지에서 CSV 파일을 다운로드 후 사용할 수 있습니다.

가장 최신 6개월의 자료인 2022년 7월 ~ 2022년 12월 공공자전거 대여 이력 정보를 내려받아 받겠습니다.

이를 통해 가져온 CSV는 다음과 같습니다.

> (가져온 CSV는 data/대여이력 폴더 내부에 저장됩니다.)


In [2]:
rent_info = pd.read_csv(
    'data/대여이력/서울특별시 공공자전거 대여이력 정보_2212.csv', encoding='cp949')
rent_info.head(5)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),생년,성별,이용자종류,대여대여소ID,반납대여소ID
0,SPB-44695,2022-12-01 00:00:10,1933,개봉푸르지오아파트 상가,0,2022-12-01 00:00:20,01933,개봉푸르지오아파트 상가,0,0,0.00,\N,M,내국인,ST-678,ST-678
1,SPB-31562,2022-12-01 00:00:04,3007,MBC 앞,0,2022-12-01 00:00:26,03007,MBC 앞,0,0,111.20,1995,F,내국인,ST-2165,ST-2165
2,SPB-56324,2022-12-01 00:01:33,4468,가락1동주민센터,0,2022-12-01 00:01:58,04468,가락1동주민센터,0,0,0.00,1994,M,내국인,ST-2583,ST-2583
3,SPB-30175,2022-12-01 00:03:02,652,답십리 래미안엘파인아파트 입구,0,2022-12-01 00:03:30,00652,답십리 래미안엘파인아파트 입구,0,0,0.00,1981,M,내국인,ST-1447,ST-1447
4,SPB-37639,2022-12-01 00:00:18,1047,강동 한신휴플러스,0,2022-12-01 00:03:55,01075,천동초교 삼거리,0,3,492.34,1989,M,내국인,ST-1369,ST-1836


### "서울시 공공자전거 대여소" 정보 수집

> 본 자료는 API 형식으로 제공됨으로 자료를 내려 받아 csv 파일로 변환한 다음 사용하겠습니다.

본 자료에 대한 API는 https://data.seoul.go.kr/dataList/OA-21235/S/1/datasetView.do 을 통해 발급받을 수 있습니다.

위 홈페이지에서 API key를 발급받은 후 http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/1/1000 와 같이 요청하여 사용할 수 있습니다. 이때 마지막 부분의 1/1000에서 1은 시작 페이지 번호이고, 1000은 끝 페이지 번호입니다.

한 번의 요청으로 최대 1,000개의 데이터만 받을 수 있기 때문에
1,000개 단위로 API를 요청하여 정보를 받아온 다음 하나의 CSV 파일로 합쳐주는 작업을 아래 코드를 통해 수행하겠습니다.

> CSV 파일은 data/대여소 정보.csv 라는 파일명으로 저장될 것입니다.


In [3]:
from requests import get
from csv import DictWriter

# 데이터의 key값들
keys = ['LENDPLACE_ID', 'STATN_ADDR1', 'STATN_ADDR2', 'STATN_LAT', 'STATN_LNT']
row_list = []

page_count = 1
while (True):
    # 요청 url 설정
    url = f'http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/{(page_count-1)*1000 + 1}/{page_count * 1000}'

    # OpenAPI에 요청
    print("Request: " + url)
    response = get(url)
    response.encoding = 'utf-8'
    dic = response.json()

    # 마지막 페이지일 경우 반복문을 종료한다.
    if (dic.get('bikeStationMaster') == None and dic.get('RESULT').get('CODE') == 'INFO-200'):
        break

    # 알맞은 데이터를 받았는지 확인
    status = dic['bikeStationMaster']['RESULT']['CODE']
    if (status != 'INFO-000'):
        print(f'Error: Status with code {status}')
        exit(1)

    # csv에 저장할 필요한 정보 추출(요청상태와 같이 불필요한 데이터는 csv에 저장하지 않는다)
    data = dic['bikeStationMaster']['row']
    row_list = row_list + data
    page_count = page_count + 1

# 추출된 정보들을 csv 파일로 저장
save_csv_path = 'data/대여소 정보.csv'
with open(save_csv_path, 'w', encoding='UTF-8', newline='') as f:
    print("Saving with csv...")
    w = DictWriter(f, keys)
    w.writeheader()
    w.writerows(row_list)

print(f'Save file: "{save_csv_path}"')

Request: http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/1/1000
Request: http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/1001/2000
Request: http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/2001/3000
Request: http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/3001/4000
Request: http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/4001/5000
Saving with csv...
Save file: "data/대여소 정보.csv"


API를 통해 가져온 정보를 확인해 보겠습니다.


In [4]:
location = pd.read_csv('data/대여소 정보.csv', encoding='UTF-8')
location.head(5)

,LENDPLACE_ID,STATN_ADDR1,STATN_ADDR2,STATN_LAT,STATN_LNT
0,ST-10,서울특별시 마포구 양화로 93,427,37.552746,126.918617
1,ST-100,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.536667,127.073593
2,ST-1000,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.510380,126.866798
3,ST-1001,서울특별시 양천구 남부순환로4길20,서서울호수공원,0.000000,0.000000
4,ST-1002,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.529900,126.876541


### "종관기상관측(ASOS)" 자료 수집

본 데이터 세트의 https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=1 홈페이지에서 CSV 파일을 내려받은 후 사용할 수 있습니다.

서울시의 2022년 각 날짜에 따른 날씨 데이터로 선택하여 내려받았습니다.

이를 통해 가져온 CSV는 다음과 같습니다.

> (가져온 CSV는 data/seoul_weather.csv 로 저장됩니다.)


In [5]:
weather = pd.read_csv('data/seoul_weather_2022.csv', encoding='cp949')
weather.head(5).T

,0,1,2,3,4
지점,108,108,108,108,108
일시,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05
평균기온(°C),-4.3,-1.3,-1.9,-2.5,-2.8
최저기온(°C),-10.2,-5.2,-8.0,-5.6,-7.8
최저기온 시각(hhmi),710.0,2356.0,714.0,2400.0,634.0
최고기온(°C),2.3,3.0,2.5,1.0,1.9
최고기온 시각(hhmi),1544,1551,1542,1445,1518
강수 계속시간(hr),NaN,4.17,4.0,0.92,NaN
10분 최다 강수량(mm),NaN,NaN,NaN,NaN,NaN
10분 최다강수량 시각(hhmi),NaN,NaN,NaN,NaN,NaN


## 데이터 전처리

대여소에 대한 데이터와 대여 이력에 대한 정보 모두 상당히 많은 양의 정보를 담고 있으나 **최적의 자전거 재배치 동선 설계**에는 사용되지 않는 **불필요한** 정보들이 포함되어 있습니다. 또한 정제되지 않은 순수 데이터로 Na, 이상치, 등 정확한 데이터 분석을 방해하는 요소들도 많이 있습니다.

본 단계에서는 불필요한 열의 병합 또는 제거, Na값 처리, 이상치 제거, 등의 방법을 통해 데이터를 전처리한 다음 데이터 분석을 수행할 때 바로 활용할 수 있도록 새로운 CSV 파일로 저장할 것입니다.


### "서울시 공공자전거 대여 이력 정보" 전처리

#### 데이터 분할

open API를 통해 받아온 "대여소 정보"에는 각 대여소에 주소와 위치값은 존재하지만, 대여소 이름, 대여소 번호, 대여소 거치대 개수와 같은 세부적인 정보를 불포함하고 있습니다.

공공데이터 포털에서 "대여소의 세부적인 정보"를 포함하고 있는 정보를 찾아보았으나 대여소의 위치(위도, 경도)와 세부 정보를 동시에 제공하지 않았습니다.

이에 "서울시 공공자전거 대여 이력 정보" 파일에 있는 대여소 번호, 대여소명, 대여소 거치대 개수를 추출하여 "대여소 정보" 파일에 합쳐주도록 하겠습니다. 본 과정은 아래 순서로 진행됩니다.

1. 대여이력 CSV 파일을 읽어와 "대여대여소ID", "대여 대여소번호", "대여 대여소명", "대여거치대" 열의 값을 가져옵니다.

2. 각 열의 이름을 '대여소ID', '대여소번호', '대여소명', '거치대'로 변경합니다.

3. 대여이력 CSV 파일을 읽어와 "반납반납소ID", "반납 반납소번호", "반납 반납소명", "반납거치대" 열의 값을 가져옵니다.

4. 각 열의 이름을 '대여소ID', '대여소번호', '대여소명', '거치대'로 변경합니다.

5. 위의 두 데이터프레임을 하나로 통합합니다. 통합할 때 대여소ID가 중복되는 값들은 제거합니다.

6. 대여소ID가 결측값을 제거한 뒤 "대여소ID"를 인덱스로 설정하여 빠르게 값을 찾을 수 있도록 합니다.

7. locationInfo 폴더에 하나의 CSV 파일로 저장한다. 파일명은 2212.csv 와 같이 년도와 월로 합니다.

8. 위 과정을 모든 대여이력 CSV 파일에 대해 수행합니다.

9. locationInfo 폴더에 생성된 모든 대여소 세부 정보 CSV 파일들을 불러옵니다.

10. 대여소ID값이 중복될 경우 제거합니다.

11. "대여소 정보.csv" 파일을 가져와 대여소ID값을 기준으로 대여소 세부 정보들을 결합(join)합니다.

12. 변경된 정보를 파일에 덮어씁니다.

8번 이후의 과정은 모든 파일에 대해 처리를 완료한 다음 진행해야 하니 우선 하나의 파일에 대해서 7번 과정까지 진행해 보겠습니다.

> 대여이력 csv파일은 결측값에 대해 "\N" 문자열이 입력되어 있습니다(혹은 pandas가 CSV를 읽어올 때 결측값에 대해 줄내림문자(\N)로 인식했을 수도 있습니다). 이에 \N 값을 Na로 변환하여 제거하겠습니다.


In [6]:
# 대여이력 정보를 불러온다. 12월 자료에 대해 우선적으로 적용한 다음 다른 자료에도 순차적으로 적용하겠다.
rent_history_data = pd.read_csv(
    'data/대여이력/서울특별시 공공자전거 대여이력 정보_2212.csv', encoding='cp949')

# 필요한 열의 데이터만 가져온다.
location_data = rent_history_data.loc[:, [
    '대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]

# 대여소ID에 따른 대여소 정보를 가져온 뒤 열 이름을 통일해준다.
rent_location = location_data[['대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대']]
return_location = location_data[['반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]
rent_location.columns = return_location.columns = ['대여소ID', '대여소번호', '대여소명', '거치대']

# \N값 제거, 결측값 제거, 대여소ID 중복값 제거, 대여소ID를 인덱스로 변환하는 작업을 수행한다.
location_info = (pd.concat([rent_location, return_location])
                 .replace(r"\\N", np.nan, regex=True)
                 .dropna(subset=['대여소ID'])
                 .drop_duplicates(subset="대여소ID")
                 .set_index("대여소ID"))

# csv로 저장
location_info.to_csv('data/locationInfo/2212.csv', encoding='UTF-8-sig')
print("Done")

Done


저장된 CSV는 다음과 같습니다.

> 거치대 숫자가 0인 것은 정상적인 정보가 아니지만 공공자전거는 거치대의 개수와 관계없이 위치기반으로 반납을 처리하며 자전거를 반납할 때 항상 거치대에 거치를 이용하는 것이 아니기에 해당 오류는 무시합니다.


In [7]:
location_info = pd.read_csv('data/locationInfo/2212.csv', encoding='UTF-8-sig')
location_info.head(5)

,대여소ID,대여소번호,대여소명,거치대
0,ST-678,1933,개봉푸르지오아파트 상가,0
1,ST-2165,3007,MBC 앞,0
2,ST-2583,4468,가락1동주민센터,0
3,ST-1447,652,답십리 래미안엘파인아파트 입구,0
4,ST-1369,1047,강동 한신휴플러스,0


#### 사용하지 않는 열 제거

- 본 연구의 목적은 "최적의 자전거 재배치 동선 설계"로 생년월일, 성별, 이용자 종류, 등 이용자에 대한 세부정보는 사용되지 않습니다. 따라서 해당 열들을 제거해 주겠습니다.

- "데여소 정보"에 통합하기 위해 별도의 CSV 파일로 분리하였던 '대여 대여소번호', '대여 대여소명', '대여거치대', '반납대여소번호', '반납대여소명', '반납거치대' 열 또한 제거합니다.

- 각 자전거에 따른 구분을 하는 것이 아니기에 "자전거 번호" 열 또한 제거합니다.

위 과정은 결론적으로 대여이력 정보에서 '대여일시', '반납일시', '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID' 열만 가져오는 것과 동일합니다. 이에 아래와 같이 작성될 수 있습니다.


In [8]:
use_data = rent_history_data.loc[:, [
    '대여일시', '반납일시', '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID']]

use_data.info()
use_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815390 entries, 0 to 1815389
Data columns (total 6 columns):
 #   Column   Dtype  
---  ------   -----  
 0   대여일시     object 
 1   반납일시     object 
 2   이용시간(분)  int64  
 3   이용거리(M)  float64
 4   대여대여소ID  object 
 5   반납대여소ID  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 83.1+ MB


,대여일시,반납일시,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID
0,2022-12-01 00:00:10,2022-12-01 00:00:20,0,0.00,ST-678,ST-678
1,2022-12-01 00:00:04,2022-12-01 00:00:26,0,111.20,ST-2165,ST-2165
2,2022-12-01 00:01:33,2022-12-01 00:01:58,0,0.00,ST-2583,ST-2583
3,2022-12-01 00:03:02,2022-12-01 00:03:30,0,0.00,ST-1447,ST-1447
4,2022-12-01 00:00:18,2022-12-01 00:03:55,3,492.34,ST-1369,ST-1836


#### 이상치 제거

정상적이지 않은 값들은 정확한 데이터 분석을 방해합니다. 따라서 아래 과정을 통해 이상치를 제거하겠습니다.

1. 본 데이터셋의 모든 값은 필수적으로 존재해야 함으로 NA값을 모두 제거하겠습니다.

2. 이용시간이 0보다 작거나 1440(24시간)보다 클 경우 제거합니다.

3. 이용거리가 0보다 작거나 36780(서울 가로 횡단 거리)보다 클 경우 제거합니다.

4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거합니다.<br>이후 처리가 완료된 speed 열은 제거합니다.

> 본 데이터셋(12월)에 대해서는 변경된 데이터를 기존 데이터에 덮어쓰는 방식으로 코드를 작성하였습니다.<br> 하지만 이후 OS 모듈을 이용한 모든 파일에 적용할 때는 파이프라인(pipe function) 함수를 이용하여 작성하였습니다.


##### 1. NA값 제거

본 데이터셋의 모든 값은 필수적으로 존재해야 함으로 NA값을 모두 제거하겠습니다.


In [9]:
# 1. na값을 모두 제거한다.
use_data = use_data.dropna()

##### 2. 이용시간 이상치 제거

이용 시간은 하루 단위로 측정됨으로 0 초과 24 이하의 값을 가져야 합니다.

따라서 이용 시간이 0 이하인 경우와 24시간(1,440분) 초과면 이상치로 판단하여 제거합니다.


In [10]:
# 2. 이용시간이 0이하, 1440초과인 값을 가지면 제거한다.
use_data = use_data[use_data['이용시간(분)'].between(1, 1440)]

##### 3. 이용거리 이상치 제거

서울시 공공자전거 사업은 서울시에서 운영하는 공유 정책 사업으로 공공자전거 대여소는 서울시 지역 내부에만 존재합니다.

따라서 이용거리가 0이하이거나 서울의 가로 횡단 길이에 해당하는 36780m 초과이면 이상치라고 판단하여 제거합니다.


In [11]:
# 3. 이용거리가 0 이하, 36780 초과인 값을 가지면 제거한다.
use_data = use_data[use_data['이용거리(M)'].between(1, 36780)]

##### 4. 이동속도 이상치 제거

이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거합니다.

작업을 수행한 이후, 처리가 완료된 speed 열은 제거합니다.


In [12]:
# 4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거한다.
# 속도(m/min)를 구하고 계산 결과 발생한 inf 값에 대해 제거한다.
use_data = use_data.assign(speed=use_data['이용거리(M)'] / use_data['이용시간(분)'])
use_data = use_data.replace([np.inf, -np.inf], np.nan).dropna()
use_data = use_data[use_data.speed < 666]

# 처리가 완료된 speed 열은 제거한다.
use_data = use_data.drop(columns='speed')

#### CSV로 저장

위 과정을 통해 전처리를 완료한 데이터프레임은 데이터 사용시 바로 사용할 수 있도록 csv 파일로 저장합니다.


In [13]:
# csv 파일로 저장한다.
use_data.to_csv(f'data/preprocessing/2212.csv', index=False)
print(f'Save: data/preprocessing/2212.csv\n')

Save: data/preprocessing/2212.csv



#### 각 파일에 적용

위 "서울시 공공자전거 대여 이력 정보" 전처리 과정을 각각의 파일에 대해 수행하였습니다.


In [14]:
def preprocess_csv(file_name, save_file_name, data_path):
    data = pd.read_csv(f'{data_path}/대여이력/{file_name}', encoding='cp949')

    # 필요한 열의 데이터만 가져온다.
    location_data = data.loc[:, ['대여대여소ID', '대여 대여소번호', '대여 대여소명',
                                 '대여거치대', '반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]

    # 대여소ID에 따른 대여소 정보를 가져온 뒤 열 이름을 통일해준다.
    rent_location = location_data[['대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대']]
    return_location = location_data[['반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]
    rent_location.columns = return_location.columns = ['대여소ID', '대여소번호', '대여소명', '거치대']

    # \N값 제거, 결측값 제거, 대여소ID 중복값 제거, 대여소ID를 인덱스로 변환하는 작업을 수행한다.
    location_info = (pd.concat([rent_location, return_location])
                     .replace(r"\\N", np.nan, regex=True)
                     .dropna(subset=['대여소ID'])
                     .drop_duplicates(subset="대여소ID")
                     .set_index("대여소ID"))

    # csv로 저장
    location_info.to_csv(
        f'{data_path}/locationInfo/{save_file_name}.csv', encoding='UTF-8-sig')
    print(f'Save: {data_path}/locationInfo/{save_file_name}.csv')

    use_data = data.loc[:, ['대여일시', '반납일시',
                            '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID']]

    use_data = (
        # 1. na값을 모두 제거한다.
        use_data.dropna()

        # 2. 이용시간이 0이하, 1440초과인 값을 가지면 제거한다.
        .pipe(lambda df: df[df['이용시간(분)'].between(1, 1440)])

        # 3. 이용거리가 0 이하, 36780 초과인 값을 가지면 제거한다.
        .pipe(lambda df: df[df['이용거리(M)'].between(1, 36780)])

        # 4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거한다.
        # 속도(m/min)를 구하고 계산 결과 발생한 inf 값에 대해 제거한다.
        .pipe(lambda df: df.assign(speed=df['이용거리(M)'] / df['이용시간(분)']))
        .pipe(lambda df: df.replace([np.inf, -np.inf], np.nan).dropna())
        .pipe(lambda df: df[df.speed < 666])

        # 처리가 완료된 speed 열은 제거한다.
        .pipe(lambda df: df.drop(columns='speed'))
    )

    # csv 파일로 저장한다.
    use_data.to_csv(f'{data_path}/preprocessing/{save_file_name}.csv', index=False)
    print(f'Save: {data_path}/preprocessing/{save_file_name}.csv\n')


# 사용할 파일들을 리스트업한다.
file_list = []
file_dir = os.path.join(data_path, '대여이력')
for file_name in os.listdir(file_dir):
    if (os.path.splitext(file_name)[1] == '.csv'):
        file_list.append(file_name)

# 폴더가 없을 경우 생성한다.
if not os.path.exists(location_info_path):
    os.mkdir(location_info_path)
if not os.path.exists(preprocessing_path):
    os.mkdir(preprocessing_path)

for file_name in file_list:
    # 파일의 날짜 형식이 22.01과 _2211 두가지로 존재한다.
    # .과 _를 제거하여 날짜형식을 2211과 같이 통일한다.
    save_file_name = (file_name[-9:]
                      .replace('.', '')
                      .replace('_', '')
                      .replace('csv', ''))
    print(save_file_name, file_name)

    # csv 처리를 한다.
    preprocess_csv(file_name, save_file_name, data_path)
print(f'A total of {len(file_list)} csv files were processed.')

2207 서울특별시 공공자전거 대여이력 정보_2207.csv
Save: /Users/jaewone/Downloads/공공자전거/data/locationInfo/2207.csv
Save: /Users/jaewone/Downloads/공공자전거/data/preprocessing/2207.csv

2212 서울특별시 공공자전거 대여이력 정보_2212.csv
Save: /Users/jaewone/Downloads/공공자전거/data/locationInfo/2212.csv
Save: /Users/jaewone/Downloads/공공자전거/data/preprocessing/2212.csv

2210 서울특별시 공공자전거 대여이력 정보_2210.csv
Save: /Users/jaewone/Downloads/공공자전거/data/locationInfo/2210.csv
Save: /Users/jaewone/Downloads/공공자전거/data/preprocessing/2210.csv

2211 서울특별시 공공자전거 대여이력 정보_2211.csv
Save: /Users/jaewone/Downloads/공공자전거/data/locationInfo/2211.csv
Save: /Users/jaewone/Downloads/공공자전거/data/preprocessing/2211.csv

2208 서울특별시 공공자전거 대여이력 정보_2208.csv
Save: /Users/jaewone/Downloads/공공자전거/data/locationInfo/2208.csv
Save: /Users/jaewone/Downloads/공공자전거/data/preprocessing/2208.csv

2209 서울특별시 공공자전거 대여이력 정보_220

### "서울시 공공자전거 대여소" 전처리

#### 세부주소 통합

"대여소 정보.csv"에서 STATN_ADDR1는 메인 주소이고 STATN_ADDR2는 서브 주소입니다.

서브 주소의 경우 많은 Na값을 포함하고 있기에 Na값을 빈 문자열(`""`)로 치환한 뒤 두 열을 하나의 열로 통합하여 주겠습니다.


In [15]:
location_path = 'data/대여소 정보.csv'
location = pd.read_csv(location_path)

# na값을 빈 값으로 채운 뒤
location = location.fillna('')
location = location.assign(주소=location.STATN_ADDR1 + ' ' + location.STATN_ADDR2)

# STATN_ADDR1과 STATN_ADDR2 열 삭제
location = location.drop(columns=['STATN_ADDR1', 'STATN_ADDR2'])
location.head(5)

# Na값이 0개 인 것을 확인 할 수 있다.
print(f'Location data contain {location.isna().sum().sum()} na values')

Location data contain 0 na values


#### 추가 정보 통합

"서울시 공공자전거 대여 이력" 정보의 전처리 과정에서 분할하였던 대여소에 대한 데이터를 "대여소ID"값을 기준으로 통합해 주겠습니다.

통합하는 과정은 아래와 같습니다.

1. locationInfo 폴더 내부에 저장된 대여소 정보들을 모두 읽어옵니다.

2. 대여소ID 열을 기준으로 중복으로 제거합니다.

3. "대여소 정보.csv"에서 대여소ID값을 기준으로 두 테이블을 결합합니다.

4. 열의 이름을 한글로 통일합니다.


In [16]:
# 폴더의 경로를 설정한다.
common_table = pd.DataFrame(columns=['대여소ID', '대여소번호', '대여소명', '거치대'])

# 각 csv를 읽어와 대여소ID를 기준으로 중복된 열을 제거한다.
for file in os.listdir(location_info_path):
    file_path = os.path.join(location_info_path, file)
    if (os.path.splitext(file_path)[1] == '.csv'):
        table = pd.read_csv(file_path, encoding='UTF-8')
        if (len(common_table) == 0):
            common_table = table
        else:
            common_table = (pd.concat(
                [common_table, table], ignore_index=True).drop_duplicates(subset="대여소ID"))

# 대여소ID를 index로 설정한다.
common_table = common_table.set_index('대여소ID', drop=True)

# LENDPLACE_ID와 대여소ID값을 기준으로 LEFT JOIN을 수행한다.
location = pd.merge(location, common_table,
                    left_on='LENDPLACE_ID', right_on='대여소ID', how='left')

# 열의 이름을 한글로 통일해준다.
location = location.rename(
    columns={'LENDPLACE_ID': '대여소ID', 'STATN_LAT': '위도', 'STATN_LNT': '경도'})

# 필수적으로 필요한 열(대여소ID, 위도, 경도)은 Na값이 없는 것은 확인 할 수 있다.
print(
    f'Location data contain {location.iloc[:,0:3].isna().sum().sum()} na values')

# 올바르게 합쳐졌음을 확인 할 수 있다.
location.head(5)

Location data contain 0 na values


,대여소ID,위도,경도,주소,대여소번호,대여소명,거치대
0,ST-10,37.552746,126.918617,서울특별시 마포구 양화로 93 427,108.0,서교동 사거리,0.0
1,ST-100,37.536667,127.073593,서울특별시 광진구 아차산로 262 더샵스타시티 C동 앞,NaN,NaN,NaN
2,ST-1000,37.510380,126.866798,서울특별시 양천구 신정동 236 서부식자재마트 건너편,729.0,서부식자재마트 건너편,0.0
3,ST-1001,0.000000,0.000000,서울특별시 양천구 남부순환로4길20 서서울호수공원,NaN,NaN,NaN
4,ST-1002,37.529900,126.876541,서울특별시 양천구 목동동로 316-6 서울시 도로환경관리센터,731.0,서울시 도로환경관리센터,0.0


#### 위도와 경도 값의 이상치 제거

데이터 세트의 설명란에서 위도와 경도의 값이 0인 대여소(행)는 더 이상 사용되지 않는 대여소라고 하였습니다.

따라서 위도와 경도 값이 0인 행들을 찾아 모두 제거하였습니다.


In [17]:
# 경도와 위도의 좌표가 0인 값이 총 77개가 존재한다.
(location[['위도', '경도']] != 0.0).all(True).value_counts()

True     3144
False      77
dtype: int64

In [18]:
# 경도와 위도의 좌표가 0인 행들을 모두 제거한다.
location = location[(location[['위도', '경도']] != 0.0).all(True)]
(location[['위도', '경도']] != 0.0).all(True).value_counts()

True    3144
dtype: int64

#### NA값 처리

추가 정보를 통합한 결과 위도와 경도값이 0인 행에서 거치대의 개수가 Na인 것을 확인 할 수 있습니다.

해당값들의 존재 유무는 크게 영향을 받지 않기에 0으로 채워주겠습니다.


In [19]:
# 거치대 열의 결측값을 채워준다.
location['거치대'] = location['거치대'].fillna(0)

#### CSV로 저장

전처리가 완료된 값으로 "대여소 정보.csv"를 덮어씌우겠습니다.


In [20]:
# csv로 저장한다.
location.to_csv(location_path, index=False, mode='w', encoding='UTF-8-sig')

### "종관기상관측(ASOS)" 전처리

"종관기상관측(ASOS)" 자료 중 가설을 설정하고 해당 가설에 해당하는 정보만을 추출하겠습니다.


#### 가설 설정 및 정보 추출

1.  **기온이 지나치게 낮거나 지나치게 높을 경우 공공자전거 이용율이 감소할 것이다.**

    전체 평균기온의 평균기온을 기준으로 차를 계산합니다.

2.  **일교차가 클 경우 공공자전거 이용률이 감소할 것이다.**

    최고기온에서 최저기온을 빼서 일교차 값을 구합니다.

3.  **비가 오면 공공자전거 이용률이 감소할 것이다.**

    '강수 계속시간(hr)' 값으로 강수시간과 공공자전거 이용률 간의 상관관계를 분석합니다.

    > 본 데이터 세트에서 강수와 공공자전거 이용률 간의 상관관계는 '일강수량(mm)' 값을 통해 분석하는 것이 더 타당한 것으로 판단됩니다. <br>하지만 '강수 계속시간(hr)'이 0 이상(비가 왔다)임에도 '일강수량(mm)' 값이 0인 오류를 내포하고 있으므로 '강수 계속시간(hr)'이 더 올바르게 측정된 자료라고 판단할 수 있습니다. <br>이에 '일 강수량(mm)' 대신 '강수 계속시간(hr)'값을 통해 강수와 공공자전거 이용률 간의 상관관계를 분석합니다.

4.  **눈이 오면 공공자전거 이용율이 감소할 것이다.**

    '일 최심적설(cm)' 값으로 적설량과 공공자전거 이용률 간의 상관관계를 분석합니다.

5.  **안개로 인하여 가시거리가 축소되면 공공자전거 이용률이 감소할 것이다.**

    '안개 계속시간(hr)' 값의 존재여부에 따라 안개가 발생하였다고 판단합니다.

    > 안개 또한 공공자전거 이용율에 영향을 미칠 것으로 판단되나 본 데이터 세트에서 '안개 계속시간(hr)'이 1개의 값만을 가지고 있으므로 유연한 판단이 어렵습니다. 따라서 **안개에 따른 영향은 고려하지 않습니다.**


In [21]:
# 2022년 7월부터 2022년 12월 사이의 공공자전거 대여 이력 정보를 사용하기에
# 날짜를 인덱스로 하여 7월부터 12월 사이의 날짜를 가져온다.
weather = pd.read_csv(
    'data/seoul_weather_2022.csv',
    encoding='cp949',
    index_col="일시"
).loc['2022-07-01':'2022-12-31']


# 사용할 정보들을 담을 새 데이터프레임을 정의한다.
use_weather = pd.DataFrame(index=weather.index)

##### 1. 기온이 지나치게 낮거나 지나치게 높을 경우 공공자전거 이용율이 감소할 것이다

전체 평균기온의 평균기온을 기준으로 차를 계산합니다.


In [22]:
# 평균기온 열에 Na 값이 없는 것을 확인 할 수 있다.
weather['평균기온(°C)'].isna().sum()  # 0

mean_weather = weather['평균기온(°C)'].mean()
use_weather['temp_10'] = abs(weather['평균기온(°C)'] - mean_weather)
use_weather.head(5).T

일시,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05
temp_10,10.399457,12.599457,13.099457,13.099457,12.999457


##### 2. 일교차가 클 경우 공공자전거 이용률이 감소할 것이다.

최고기온에서 최저기온을 빼서 일교차 값을 구합니다.

최고기온에서 최저기온을 빼었을 때 NaN 값이 하나 존재합니다. 이는 최저기온의 2022-08-08 값이 NaN이기 때문입니다.

이에 2022-08-08의 값을 앞뒤 날의 최저기온 평균값으로 대치하겠습니다.


In [23]:
gap = weather['최고기온(°C)'] - weather['최저기온(°C)']

# 최고기온에서 최저기온을 빼었을 때 NaN 값이 하나 존재한다.
gap.isna().sum()

# 조사 결과 2022-08-08에서 최저기온 값이 NaN이다.
weather[((gap >= 0) == False)]

# 2022-08-08의 값을 앞뒤날의 최저기온 평균값으로 대치한다.
gap['2022-08-08'] = (gap['2022-08-07'] + gap['2022-08-09']) / 2

# use_weather 데이터프레임에 일교차값을 추가한다.
use_weather['gap'] = gap
use_weather.head(5).T

일시,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05
temp_10,10.399457,12.599457,13.099457,13.099457,12.999457
gap,9.000000,9.800000,9.800000,6.200000,6.400000


##### 3. 비가 오면 공공자전거 이용률이 감소할 것이다.

'강수 계속시간(hr)' 값으로 강수시간과 공공자전거 이용률 간의 상관관계를 분석합니다.

> 본 데이터 세트에서 강수와 공공자전거 이용률 간의 상관관계는 '일강수량(mm)' 값을 통해 분석하는 것이 더 타당한 것으로 판단됩니다. <br>하지만 '강수 계속시간(hr)'이 0 이상(비가 왔다)임에도 '일강수량(mm)' 값이 0인 오류를 내포하고 있음으로 '강수 계속시간(hr)'이 더 올바르게 측정된 자료라고 판단할 수 있습니다. <br>이에 '일강수량(mm)' 대신 '강수 계속시간(hr)'값을 통해 강수와 공공자전거 이용률 간의 상관관계를 분석합니다.


In [24]:
# 비가 오지 않은 날은 Na로 처리되어 있다. 이를 0으로 변경하여 Na값을 처리한다.
# 강수 계속시간이 정상적인(시간임으로 0이상 24이하) 값의 분포를 가짐을 확인할 수 있다.
weather['강수 계속시간(hr)'].fillna(0).describe()

count    184.000000
mean       2.801848
std        5.100971
min        0.000000
25%        0.000000
50%        0.000000
75%        3.602500
max       24.000000
Name: 강수 계속시간(hr), dtype: float64

In [25]:
# use_weather 데이터프레임에 강수 계속시간을 추가한다.
use_weather['rain_hr'] = weather['강수 계속시간(hr)'].fillna(0)
use_weather.head(5).T

일시,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05
temp_10,10.399457,12.599457,13.099457,13.099457,12.999457
gap,9.000000,9.800000,9.800000,6.200000,6.400000
rain_hr,0.000000,0.000000,0.000000,0.830000,1.420000


##### 4. 눈이 오면 공공자전거 이용률이 감소할 것이다.

'일 최심적설(cm)' 값으로 적설량과 공공자전거 이용률 간의 상관관계를 분석합니다.


In [26]:
# 눈이 오지 않은 날은 Na로 처리되어 있다. 이를 0으로 변경하여 Na값을 처리한다.
weather['일 최심적설(cm)'].fillna(0).describe()

count    184.000000
mean       0.160326
std        0.616620
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        4.500000
Name: 일 최심적설(cm), dtype: float64

In [27]:
# use_weather 데이터프레임에 강수 계속시간을 추가한다.
use_weather['snow'] = weather['일 최심적설(cm)'].fillna(0)
use_weather.head(5).T

일시,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05
temp_10,10.399457,12.599457,13.099457,13.099457,12.999457
gap,9.000000,9.800000,9.800000,6.200000,6.400000
rain_hr,0.000000,0.000000,0.000000,0.830000,1.420000
snow,0.000000,0.000000,0.000000,0.000000,0.000000


##### 생성된 데이터프레임 확인

위 과정을 통해 새성된 use_weather 데이터프레임을 확인합니다.


In [28]:
print(use_weather.head(5).T)
use_weather.describe().T

일시       2022-07-01  2022-07-02  2022-07-03  2022-07-04  2022-07-05
temp_10   10.399457   12.599457   13.099457   13.099457   12.999457
gap        9.000000    9.800000    9.800000    6.200000    6.400000
rain_hr    0.000000    0.000000    0.000000    0.830000    1.420000
snow       0.000000    0.000000    0.000000    0.000000    0.000000


,count,mean,std,min,25%,50%,75%,max
temp_10,184.0,9.177658,5.986797,0.099457,4.774457,8.599457,12.099457,28.000543
gap,184.0,8.173913,2.761815,2.400000,6.300000,8.050000,10.125000,16.600000
rain_hr,184.0,2.801848,5.100971,0.000000,0.000000,0.000000,3.602500,24.000000
snow,184.0,0.160326,0.616620,0.000000,0.000000,0.000000,0.000000,4.500000


#### 이상치 조정

이상치를 적절하게 조절하여 지나치게 크거나 동떨어진 데이터가 데이터 분포에 영향을 주는 것을 방지합니다.


##### 1. 강수시간 이상치 제거

use_weather 데이터프레임의 describe를 확인할 수 있듯 강수시간(rain_hr)은 75% 이상의 값이 3 이하의 값을 가지는데 최댓값은 24임을 확인할 수 있습니다.

이처럼 평균에서 지나치게 떨어진 값은 분포에 영향을 미칠 수 있음으로 적절한 임계값으로 조정해 주겠습니다.


In [29]:
rain_hr = use_weather['rain_hr']

rain_hr.value_counts().sort_index().index

# 약 10%의 데이터들이 10보다 큰 값(10이상 24이하)을 가지고 있다.
print(f'Over 10 value counts: {rain_hr[rain_hr > 10].count()}')  # 30

# 10보다 큰 값들은 모두 10로 변경한다.
rain_hr[rain_hr > 10] = 10

use_weather['rain_hr'].describe()

Over 10 value counts: 18


count    184.000000
mean       2.202826
std        3.366124
min        0.000000
25%        0.000000
50%        0.000000
75%        3.602500
max       10.000000
Name: rain_hr, dtype: float64

#### CSV로 저장

위 과정을 통해 생성된 use_weather 데이터프레임을 확인한 뒤, csv로 저장합니다.


In [30]:
print(use_weather.head(5).T)
use_weather.describe().T

일시       2022-07-01  2022-07-02  2022-07-03  2022-07-04  2022-07-05
temp_10   10.399457   12.599457   13.099457   13.099457   12.999457
gap        9.000000    9.800000    9.800000    6.200000    6.400000
rain_hr    0.000000    0.000000    0.000000    0.830000    1.420000
snow       0.000000    0.000000    0.000000    0.000000    0.000000


,count,mean,std,min,25%,50%,75%,max
temp_10,184.0,9.177658,5.986797,0.099457,4.774457,8.599457,12.099457,28.000543
gap,184.0,8.173913,2.761815,2.400000,6.300000,8.050000,10.125000,16.600000
rain_hr,184.0,2.202826,3.366124,0.000000,0.000000,0.000000,3.602500,10.000000
snow,184.0,0.160326,0.616620,0.000000,0.000000,0.000000,0.000000,4.500000


In [31]:
use_weather.to_csv('data/weather.csv')
print('Save to csv: data/weather.csv')

Save to csv: data/weather.csv


## 마무리

이번 중간보고서의 목표로 "데이터 분석 시 바로 사용할 수 있는 가용적인 상태로 만들기 위해 데이터 전처리를 수행"을 설정하였고 위 과정을 통해 수행하였습니다.

데이터 분석 연구에서 데이터 전처리에 오랜 시간을 투자하는 이유는 우선 현실에서 존재하는 데이터들은 우리가 쓰고자 하는 분석 목적에 맞게 정리되어 있지 않기 때문에 원하는 데이터들만 활용하기 위해서 전처리 작업이 필요합니다.

또한 데이터에 있어서 불필요한 정보들을 제거함으로써 더 효율적인 분석이 가능하고 모델의 성능을 향상할 수 있습니다.

모든 연구의 시작인 만큼 데이터 전처리에 대한 준비에 오랜 시간이 활용되게 됩니다.


본 과정을 통해 생성된 전처리된 "대여 이력 정보"는 다음과 같이 구성됩니다.


In [32]:
pd.read_csv('data/preprocessing/2212.csv').head(3)

,대여일시,반납일시,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID
0,2022-12-01 00:00:18,2022-12-01 00:03:55,3,492.34,ST-1369,ST-1836
1,2022-12-01 00:00:09,2022-12-01 00:04:06,3,911.71,ST-1298,ST-2268
2,2022-12-01 00:00:06,2022-12-01 00:04:12,4,660.88,ST-1512,ST-516


전처리된 "대여소 정보"는 다음과 같이 구성됩니다.


In [33]:
pd.read_csv('data/대여소 정보.csv', index_col=0).head(3)

,위도,경도,주소,대여소번호,대여소명,거치대
대여소ID,,,,,,
ST-10,37.552746,126.918617,서울특별시 마포구 양화로 93 427,108.0,서교동 사거리,0.0
ST-100,37.536667,127.073593,서울특별시 광진구 아차산로 262 더샵스타시티 C동 앞,NaN,NaN,0.0
ST-1000,37.510380,126.866798,서울특별시 양천구 신정동 236 서부식자재마트 건너편,729.0,서부식자재마트 건너편,0.0


전처리된 "종관기상관측(ASOS)"는 다음과 같이 구성됩니다.


In [34]:
pd.read_csv('data/weather.csv', index_col=0).head(5).T

일시,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05
temp_10,10.399457,12.599457,13.099457,13.099457,12.999457
gap,9.000000,9.800000,9.800000,6.200000,6.400000
rain_hr,0.000000,0.000000,0.000000,0.830000,1.420000
snow,0.000000,0.000000,0.000000,0.000000,0.000000


**팀원**

201904008 곽재원

202104216 백종민

<br>

페이지 끝. 감사합니다.
